In [6]:
import re
import csv
import pdb

import requests 
from bs4 import BeautifulSoup
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client.imdb
coll = db.test

nconst = "nm0000487"
# nconst = "nm0000229"
# nconst = "nm3915784"
# nconst = "nm4511652"
DOMAIN = "http://www.imdb.com"

url = DOMAIN + "/name/" + nconst
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html5lib')

nminfo = {}
nminfo['nconst'] = nconst
nminfo['STARmeter'] = int((soup.select_one('#meterRank').text.strip().split(" "))[-1])
nminfo['award']=soup.select_one('span[itemprop="awards"]')
if (nminfo['award'] is not None):
    nminfo['award']=' '.join(nminfo['award'].text.split())

nminfo['filmography'] = []
i=0
for filmo in soup.find('div', id='jumpto').find_all('a'):
    data = {}
    data['category'] = filmo["data-category"]
    data_id = data['category'] + "-tt"
    data['titles'] = []
    for film in soup.select('div[id^='+data_id+']'):
        tconst = film['id'].replace(data['category']+"-","")
        data['titles'].append(tconst)
    nminfo['filmography'].append(data)
    i += 1
if i==0:
    nminfo['filmography'] = None
    
nminfo['akas']=soup.find('div', id='details-akas')
if (nminfo['akas'] is not None):
    nminfo['akas'] = nminfo['akas'].text.replace(" | ",",").replace("Alternate Names:","").strip().split(",")
    
result = coll.insert_one(nminfo)
client.close()

In [9]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client.imdb
coll = db.test
cursor = coll.find()
for document in cursor:
    print(document)
client.close()

In [1]:
import re
import csv
import pdb

import requests 
from bs4 import BeautifulSoup

nconst = "nm0000487"
# nconst = "nm0000229"
# nconst = "nm3915784"
# nconst = "nm4511652"
DOMAIN = "http://www.imdb.com"

url = DOMAIN + "/name/" + nconst
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html5lib')

nminfo = {}
nminfo['nconst'] = nconst
nminfo['STARmeter'] = int((soup.select_one('#meterRank').text.strip().split(" "))[-1])
nminfo['award']=soup.select_one('span[itemprop="awards"]')
if (nminfo['award'] is not None):
    nminfo['award']=' '.join(nminfo['award'].text.strip())

nminfo['filmography'] = []
i=0
for filmo in soup.find('div', id='jumpto').find_all('a', limit=3):
    data = {}
    data['category'] = filmo["data-category"]
    data_id = data['category'] + "-tt"
    data['titles'] = []
    for film in soup.select('div[id^='+data_id+']'):
        tconst = film['id'].replace(data['category']+"-","")
        data['titles'].append(tconst)
    nminfo['filmography'].append(data)
    i += 1
if i==0:
    nminfo['filmography'] = None
    
nminfo['akas']=soup.find('div', id='details-akas')
if (nminfo['akas'] is not None):
    nminfo['akas'] = nminfo['akas'].text.replace(" | ",",").replace("Alternate Names:","").strip().split(",")

In [2]:
nminfo

{'STARmeter': 5000,
 'akas': ['An Lee', 'An Li'],
 'award': 'Won 2 Oscars.',
 'filmography': [{'category': 'director',
   'titles': ['tt3542810',
    'tt2513074',
    'tt0454876',
    'tt1127896',
    'tt0808357',
    'tt0388795',
    'tt0286716',
    'tt4290888',
    'tt0283994',
    'tt0190332',
    'tt0134154',
    'tt0119349',
    'tt0114388',
    'tt0111797',
    'tt0107156',
    'tt0105652',
    'tt3311280']},
  {'category': 'producer',
   'titles': ['tt3542810',
    'tt2513074',
    'tt2568204',
    'tt0454876',
    'tt1127896',
    'tt0808357',
    'tt0359774',
    'tt0190332',
    'tt0119349',
    'tt0114440',
    'tt0107156',
    'tt0105652']},
  {'category': 'writer',
   'titles': ['tt0255653',
    'tt0114440',
    'tt0111797',
    'tt0107156',
    'tt0105652']}],
 'nconst': 'nm0000487'}

In [ ]:
soup.find('div', id='jumpto').find_all('a', limit=3)

In [ ]:
soup.find('div', id='jumpto').find_all('a')[0]["data-category"]

In [ ]:
soup.select('div[id^="director-tt"]')[0]['id'].replace("director-","")

In [ ]:
soup.select_one('#filmo-head-director')

In [19]:
soup.select_one('div[id="details-akas"]').text.replace("\n        ", " ").strip()

'Alternate Names:     An Lee | An Li'

In [39]:
re.findall(r'(\w+)<span>|</span>(\w+)', soup.find('div', id='details-akas').text)

[]

In [3]:
soup.find('div', id='details-akas').text

'\n            Alternate Names:\n            An Lee | An Li\n        '

In [97]:
soup.find('div', id='details-akas').text.replace(" | ",",").replace("Alternate Names:","").strip().split(",")

['An Lee', 'An Li']